In [8]:
import random
seed_val = 1000
random.seed(seed_val)
import numpy as np
np.random.seed(seed_val)
import tensorflow as tf
tf.set_random_seed(seed_val)
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, TimeDistributed, SimpleRNN
from keras.layers import Concatenate, Flatten
from keras.layers import GRU, Conv2D, MaxPooling2D, AveragePooling2D, AvgPool2D, MaxPool1D
from keras.layers import Input, Reshape, Dot, Add
from keras.models import Model
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras import regularizers
from keras.utils.vis_utils import plot_model
import keras
import keras.backend as K
from data_handle import *
from gensim_wrapper import *
from utils import *
import gensim
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

In [2]:
def conv_model_multi(n_chars, n_consonant, n_vowels, n_units):
    root_word_input = Input(shape=(n_chars, n_consonant + n_vowels, 1), name="word_input")
    decoder_inputs = Input(shape=(None, n_consonant + n_vowels), name="target_consonant")

    x = Conv2D(16, (5, 5), padding='same', activation='linear')(root_word_input)
    x = AveragePooling2D(3, 3)(x)
    x = Flatten()(x)
    state_h = Dense(n_units, activation='tanh', name="state_h")(x)
    
    consonant_decoder_gru = GRU(n_units, return_sequences=True, return_state=True,  name="consonant_decoder_gru")
    consonant_decoder_outputs, _= consonant_decoder_gru(decoder_inputs, initial_state=state_h)

    vowel_decoder_gru = GRU(n_units, return_sequences=True, return_state=True, name="vowl_decoder_gru")
    vowel_decoder_outputs, _= vowel_decoder_gru(decoder_inputs, initial_state=state_h)

    consonant_decoder_dense = Dense(n_consonant, activation='softmax', name="consonant_output")
    consonant_decoder_outputs = consonant_decoder_dense(consonant_decoder_outputs)
    
    vowel_decoder_dense = Dense(n_vowels, activation='softmax', name="vowel_output")
    vowel_decoder_outputs = vowel_decoder_dense(vowel_decoder_outputs)
    
    main_model = Model([root_word_input, decoder_inputs], [consonant_decoder_outputs, vowel_decoder_outputs])
    
    encoder_model = Model(root_word_input, state_h)
    
    decoder_state_input_h = Input(shape=(n_units,))
    
    consonant_decoder_outputs, state_h= consonant_decoder_gru(decoder_inputs, initial_state=decoder_state_input_h)
    consonant_decoder_outputs = consonant_decoder_dense(consonant_decoder_outputs)
    
    vowel_decoder_outputs, state_h= vowel_decoder_gru(decoder_inputs, initial_state=decoder_state_input_h)
    vowel_decoder_outputs = vowel_decoder_dense(vowel_decoder_outputs)
    
    decoder_model = Model([decoder_inputs, decoder_state_input_h], [consonant_decoder_outputs, vowel_decoder_outputs, state_h])

    return main_model, encoder_model, decoder_model

def one_hot_sep(con, con_max, vow, vow_max):
    con_vec = np.zeros((con_max, ))
    con_vec[con] = 1
    vow_vec = np.zeros((vow_max, ))
    vow_vec[vow] = 1
    return con_vec, vow_vec
    
def decode_multi_sequence(model, char2tup, tup2char, state, n_consonant, n_vowels):
    con, vow = char2tup['&']
    con_vec, vow_vec = one_hot_sep(con, n_consonant, vow, n_vowels) 
    word_input = np.concatenate([con_vec, vow_vec]).reshape((1, 1, -1))
    decoded_chars = []
    for i in range(13):
        con_vec, vow_vec, state = model.predict([word_input, state])
        new_con_vec = np.zeros_like(con_vec)
        new_con_vec[0, 0, np.argmax(con_vec[0, 0, :])] = 1
        new_vow_vec = np.zeros_like(vow_vec)
        new_vow_vec[0, 0, np.argmax(vow_vec[0, 0, :])] = 1
        con_vec, vow_vec = new_con_vec, new_vow_vec
        word_input = np.concatenate([con_vec, vow_vec]).reshape((1, 1, -1))
        name = "{0}-{1}".format(np.argmax(con_vec[0, 0, :]), np.argmax(vow_vec[0, 0, :]))
        try:
            char = tup2char[name]
        except:
            char = ' '
        decoded_chars += [char]
    
    return decoded_chars

def pred_embeddings_multi(vocab, encoder, char2tup):
    embeddings = np.ndarray((len(vocab), embed_size))
    i = 0
    buffer = []
    buffer_size = 10000
    for wi, word in enumerate(vocab):
        word = int2word[word2int[word]]
        convec, vowvec = word2vec_seperated(char2tup, word, n_chars, n_consonant, n_vowel)
        convec = convec.reshape((-1, n_chars, n_consonant, 1))
        vowvec = vowvec.reshape((-1, n_chars, n_vowel, 1))
        mat = np.concatenate([convec, vowvec], axis=2)
        buffer.append(mat)
        if len(buffer) == buffer_size or len(vocab) - wi < buffer_size:
            buffer_np = np.stack(buffer).reshape((-1, 13, 50, 1))
            result = encoder.predict(buffer_np)
            embeddings[i:i+len(buffer)] = result
            i += len(buffer)
            buffer = []
            if i % (4 * buffer_size) == 0:
                print("Predicting: {0:.2f}%".format((i * 100.0 / len(vocab))))
                
    print("finished")
    return embeddings


In [3]:
words = read_file()
vocab, word2int, int2word = build_vocab(words)
int_words = words_to_ints(word2int, words)
word2freq = get_frequency(words, word2int, int2word)
char2int, int2char, char2tup, tup2char, n_consonant, n_vowel = build_charset()
ns_unigrams = ns_sample(word2freq, word2int, int2word, .75)
n_chars = 11 + 2 
n_features = len(char2int)
batch_size = 300
embed_size = 100
skip_window = 1
n_batches = len(vocab)  // batch_size

In [4]:
try:
    del multi_train
    del multi_enc
    del multi_dec
    keras.backend.clear_session()
    gc.collect()
except:
    pass
multi_train, multi_enc, multi_dec = conv_model_multi(n_chars, n_consonant, n_vowel, embed_size)
adam = keras.optimizers.Nadam(.001)
multi_train.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])
multi_gen = generate_word_images_multi_v5(vocab, char2tup, batch_size, n_consonant, n_vowel)

multi_train.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
word_input (InputLayer)         (None, 13, 50, 1)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 13, 50, 16)   416         word_input[0][0]                 
__________________________________________________________________________________________________
average_pooling2d_1 (AveragePoo (None, 4, 16, 16)    0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 1024)         0           average_pooling2d_1[0][0]        
__________________________________________________________________________________________________
target_con

In [5]:
history = multi_train.fit_generator(multi_gen, steps_per_epoch=n_batches, epochs = 5)

Epoch 1/5
992/992 [==============================] - 96s 96ms/step - loss: 0.7332 - consonant_output_loss: 0.4818 - vowel_output_loss: 0.2514 - consonant_output_acc: 0.8786 - vowel_output_acc: 0.9246 12s - loss: 0.8207 - consonant_output_loss: 0.5397 - vowel_output_loss: 0.2809 - conson - ETA: 9s - loss: 0.7996 - consonant_output_lo
Epoch 2/5
992/992 [==============================] - 90s 91ms/step - loss: 0.0422 - consonant_output_loss: 0.0272 - vowel_output_loss: 0.0150 - consonant_output_acc: 0.9968 - vowel_output_acc: 0.9981 3s - loss: 0.0432 - consonant_output_loss: 0.0278 - vowel_output_loss: 0.0154 - consonant_output_a
Epoch 3/5
992/992 [==============================] - 90s 91ms/step - loss: 0.0185 - consonant_output_loss: 0.0076 - vowel_output_loss: 0.0109 - consonant_output_acc: 0.9993 - vowel_output_acc: 0.9987
Epoch 4/5
992/992 [==============================] - 90s 91ms/step - loss: 0.0027 - consonant_output_loss: 0.0017 - vowel_output_loss: 9.6832e-04 - consonant_output_a

KeyboardInterrupt: 

In [6]:
embeddings = pred_embeddings_multi(vocab, multi_enc, char2tup)

Predicting: 13.43%
Predicting: 26.86%
Predicting: 40.29%
Predicting: 53.72%
Predicting: 67.15%
Predicting: 80.58%
Predicting: 94.01%
finished


In [7]:
evaluate(word2int, embeddings) 
evaluate(word2int, normalize(embeddings))

c:\users\leo\appdata\local\conda\conda\envs\gpu-tf\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


{'syntactic': 13.317757009345794,
 'semantic': 1.1070110701107012,
 'total': 8.583690987124463,
 'pick-one-out': 61.46788990825688}

In [ ]:
# file = open("results/text.txt", encoding='utf8', mode='w')
# file.write("{0} {1}\n".format(len(vocab), embed_size))
# for word, index in word2int.items():
#     e = embeddings[index]
#     e = ' '.join(map(lambda x: str(x), e))
#     file.write("{0} {1}\n".format(word, e))
# file.close()